# Federal Project Inventory Model Use
Let's use our stored (pickled) a model to identify categories given a program text (see fpi_create_model).

## Key Imports

In [1]:
import os
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline

## Define a function that will apply our estimators and use the probability to sort the categories

In [2]:
def predict_categories(text,estimators):
    """
    Given text and a dictionary of estimators, apply the estimators on the text and sort the result
    by the calculated probability
    """
    results = []
    for category, estimator in estimators.items():
        results.append((category, estimator.predict_proba([text])[0][1]))
    return sorted(results, key = lambda x: -x[1])
        

## Now load and test the estimators

In [3]:
import pickle

estimators = pickle.load(open("fpi_estimators.pkl", "rb"))

In [4]:
predict_categories("Area Development Base Program Provides access to broadband by funding for the cost of constructing, improving, and acquiring facilities and equipment for broadband service in rural communities of 20,000 inhabitants or less. Direct loans are made at the cost of money to the Treasury for the life of the facilities financed.", estimators)

[('Broadband', 1.0),
 ('Native American', 0.18675433316759743),
 ('STEM Education', 0.060783335729622176),
 ('Homelessness', 0.03067793481331821),
 ('HIV/AIDS', 0.005351769501626025),
 ('Economic Development', 0.0),
 ('Opioid Epidemic Response', 0.0),
 ('Workforce Development', 0.0),
 ('Flood Risk', 0.0),
 ('A.I. R&D/Quantum R&D', 0.0),
 ('Global Health', 0.0),
 ('Transportation Infrastructure', 0.0)]